In [184]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [245]:
def convert(val):
    val = val.replace(',', '')
    if val[0] == '(':
        val = '-' + val[1:-1]
    if val[-1] == '%':
        val = float(val[:-1]) / 100
    elif val[-1] == 'T':
        val = float(val[:-1]) * 1000000000000
    elif val[-1] == 'B':
        val = float(val[:-1]) * 1000000000
    elif val[-1] == 'M':
        val = float(val[:-1]) * 1000000
    elif val[-1] == 'K':
        val = float(val[:-1]) * 1000
    elif val == '-':
        val = 0
    else:
        val = float(val)
    return val

In [314]:
stock = "nok"
url = "https://www.marketwatch.com/investing/stock/{}/financials".format(stock)

url_add = ["/income",
       "/balance-sheet",
       "/cash-flow"]

vals = {}

quarterly = True

for url_append in url_add:
    final_url = url + url_append
    if quarterly:
        final_url += '/quarter'
    response = requests.get(final_url)
    soup = BeautifulSoup(response.text)
    rows = soup.findAll('tr', class_="table__row")
    for i in range(11, len(rows)):
        tds = rows[i].findAll('td')
        if tds == []:
            continue
        category = tds[0].div.text
        if category in vals:
            print(category)
        vals[category] = []
        for j in range(1, 6):
            vals[category].append(tds[j].text)

response = requests.get("https://www.marketwatch.com/investing/bond/tmubmusd10y?countrycode=bx")
soup = BeautifulSoup(response.text)
us_treasury_10_yr_bond_rate = float(soup.findAll('bg-quote', class_='value')[0].text) / 100

response = requests.get("https://www.marketwatch.com/investing/stock/" + stock)
soup = BeautifulSoup(response.text)
data = soup.findAll('li', class_='kv__item')
market_cap = convert(data[3].span.text[1:])
shares_outstanding = convert(data[4].span.text)
beta = float(data[6].span.text)

market_return = 0.1
perpetual_growth_rate = 1.025

Net Income Growth
Deferred Taxes
Accounts Payable
Preferred Dividends
Other Funds
Other Uses
Other Sources


In [315]:
for metric in vals:
    for i in range(len(vals[metric])):
        vals[metric][i] = convert(vals[metric][i])

In [316]:
for i in range(5):
    if vals["Free Cash Flow"][i] < 0:
        print("negative fcf")
        break

fcf_net_income_ratio = sum([vals["Free Cash Flow"][i]/vals["Net Income before Extraordinaries"][i] for i in range(5)])/5
average_revenue_growth = 1 + sum(vals["Sales Growth"][1:])/4

future_revenues = [vals["Sales/Revenue"][-1]*average_revenue_growth]
for _ in range(3):
    future_revenues.append(future_revenues[-1]*average_revenue_growth)
    
net_income_margin = sum([vals["Net Income before Extraordinaries"][i]/vals["Sales/Revenue"][i] for i in range(5)])/5
future_incomes = [net_income_margin*revenue for revenue in future_revenues]
future_free_cash_flows = [fcf_net_income_ratio*income for income in future_incomes]

cost_of_debt = 0
for i in range(5):
    debt = vals["Current Portion of Long Term Debt"][i] + vals["Long-Term Debt"][i]
    if debt != 0:
        cost_of_debt += vals["Interest Expense"][i]/debt
cost_of_debt /= 5

tax = sum([vals["Income Tax"][i]/vals["Pretax Income"][i] for i in range(5)])/5
cost_of_equity = us_treasury_10_yr_bond_rate + beta*(market_return - us_treasury_10_yr_bond_rate)

debt = vals["ST Debt & Current Portion LT Debt"][-1] + vals["Long-Term Debt"][-1]
total_cap = market_cap + debt
weight_of_debt = debt/total_cap
weight_of_equity = market_cap/total_cap

required_rate_of_return = 1 + weight_of_debt*cost_of_debt*(1-tax) + weight_of_equity*cost_of_equity

terminal_value = future_free_cash_flows[-1]*perpetual_growth_rate/(required_rate_of_return - perpetual_growth_rate)
pv_future_free_cash_flows = [future_free_cash_flows[i]/(required_rate_of_return)**(i+1) for i in range(4)]
present_terminal_value = terminal_value/(required_rate_of_return)**4
todays_value = sum(pv_future_free_cash_flows) + present_terminal_value
intrinsic_value = todays_value/shares_outstanding

print(intrinsic_value)

negative fcf
-1.0542957182493429
